# AWS SDK for Kotlin - S3 Example
This notebook uses [AWS SDK for Kotlin](https://github.com/awslabs/aws-sdk-kotlin/) to implement the basic S3 operations
- `CreateBucket`
- `PutObject`
- `ListObjectsV2`
- `GetObject`
- `DeleteBucket`

to process a csv file in `resources/sample.csv`.

Amazon S3 (Simple Storage Service) is a highly scalable object storage service from Amazon Web Services that allows you to store and retrieve any amount of data from anywhere on the web. It's commonly used for backup and restore, data archiving, content distribution, and hosting static website files, offering various storage classes with different pricing and accessibility options.

More examples can be found on [official sample repository](https://github.com/awsdocs/aws-doc-sdk-examples/tree/main/kotlin).
Free feel to contribute to add more use cases.

## Install dependencies

In [3]:
// show kernel version
"""
Kotlin Jupyter kernel version: ${notebook.kernelVersion} (Update in Settings > Tools > Kotlin Notebooks > Kernel version)
Java Runtime Environment version: ${notebook.jreInfo.javaVersion}
"""


Kotlin Jupyter kernel version: 0.15.2.706.dev1 (Update in Settings > Tools > Kotlin Notebooks > Kernel version)
Java Runtime Environment version: 21


In [1]:
%trackClasspath on
USE {
    repositories {
        mavenCentral()
    }
    dependencies {
        implementation("aws.sdk.kotlin:s3-jvm:1.4.11")
        implementation("org.jetbrains.kotlinx:dataframe:0.15.0")
    }
    import(
        "aws.sdk.kotlin.services.s3.*",
        "kotlinx.coroutines.runBlocking",
        "org.jetbrains.kotlinx.dataframe.*"
    )
}

81 new paths were added to classpath:
/Users/gaplo917/.jupyter_kotlin/maven_repository/.m2/.m2.cache/aws/sdk/kotlin/aws-config-jvm/1.4.11/aws-config-jvm-1.4.11.jar
/Users/gaplo917/.jupyter_kotlin/maven_repository/.m2/.m2.cache/aws/sdk/kotlin/aws-core-jvm/1.4.11/aws-core-jvm-1.4.11.jar
/Users/gaplo917/.jupyter_kotlin/maven_repository/.m2/.m2.cache/aws/sdk/kotlin/aws-endpoint-jvm/1.4.11/aws-endpoint-jvm-1.4.11.jar
/Users/gaplo917/.jupyter_kotlin/maven_repository/.m2/.m2.cache/aws/sdk/kotlin/aws-http-jvm/1.4.11/aws-http-jvm-1.4.11.jar
/Users/gaplo917/.jupyter_kotlin/maven_repository/.m2/.m2.cache/aws/sdk/kotlin/s3-jvm/1.4.11/s3-jvm-1.4.11.jar
/Users/gaplo917/.jupyter_kotlin/maven_repository/.m2/.m2.cache/aws/smithy/kotlin/aws-credentials-jvm/1.4.2/aws-credentials-jvm-1.4.2.jar
/Users/gaplo917/.jupyter_kotlin/maven_repository/.m2/.m2.cache/aws/smithy/kotlin/aws-event-stream-jvm/1.4.2/aws-event-stream-jvm-1.4.2.jar
/Users/gaplo917/.jupyter_kotlin/maven_repository/.m2/.m2.cache/aws/smithy/ko

## Load AWS Access Key
By default, there is a [credential chain provider](https://docs.aws.amazon.com/sdk-for-kotlin/latest/developer-guide/credential-providers.html) to look up your aws credentials from your environment.

⚠️ To remain portable and easier to understand, this example use ***custom credential provider*** that create `kotlinNotebookCredentialProvider` in `resources/aws.secret.json`

In [8]:
// Load aws.secret.json variables
%use @file[resources/aws.secret.json](currentDir=".")

In [9]:
import aws.smithy.kotlin.runtime.auth.awscredentials.Credentials
import aws.smithy.kotlin.runtime.auth.awscredentials.CredentialsProvider
import aws.smithy.kotlin.runtime.collections.Attributes

val s3Client = S3Client {
    region = "us-east-1"
    // comment this credentialsProvider if you intended to use your local environment credentials.
    credentialsProvider = kotlinNotebookCredentialProvider
}

## Create S3 Bucket
Create an empty S3 Bucket.

In [43]:
val response = runBlocking {
    s3Client.createBucket {
        bucket = "bucket-created-from-kotlin-notebook-${System.currentTimeMillis()}"
    }
}

println(response)

val bucketName = response.location?.drop(1)

println(bucketName)

CreateBucketResponse(location=/bucket-created-from-kotlin-notebook-1738687875029)
bucket-created-from-kotlin-notebook-1738687875029


## Upload a object to bucket
Upload a `resources/sample.csv` from local to S3 bucket.

In [44]:
import java.io.File

val response = runBlocking {
    s3Client.putObject {
        bucket = bucketName
        key = "sample.csv"
        body = File("resources/sample.csv").asByteStream()
    }
}

print(response)

PutObjectResponse(bucketKeyEnabled=null,checksumCrc32=6WY/lg==,checksumCrc32C=null,checksumCrc64Nvme=null,checksumSha1=null,checksumSha256=null,checksumType=FullObject,eTag="afd9b712bcb747f117167c7db28794eb",expiration=null,requestCharged=null,serverSideEncryption=Aes256,size=null,sseCustomerAlgorithm=null,sseCustomerKeyMd5=null,ssekmsEncryptionContext=*** Sensitive Data Redacted ***,ssekmsKeyId=*** Sensitive Data Redacted ***,versionId=null)

## List objects in S3 bucket
List the objects that recently uploaded to S3 bucket.

In [45]:
val response = runBlocking {
    s3Client.listObjectsV2 {
        bucket = bucketName
    }
}

println("""
Bucket: ${response.name}
Contents:
    ${response.contents?.map { "${it.key} - ${it.size} Bytes" }?.joinToString("\n")}
"""
)



Bucket: bucket-created-from-kotlin-notebook-1738687875029
Contents:
    sample.csv - 90 Bytes



## Get CSV object from S3 and render in dataframe
Get `sample.csv` from S3 and render it in dataframe format.

In [46]:
import aws.sdk.kotlin.services.s3.model.GetObjectRequest
import aws.smithy.kotlin.runtime.content.toByteArray
import aws.smithy.kotlin.runtime.content.toInputStream
import okio.internal.commonToUtf8String

runBlocking {
    s3Client.getObject(GetObjectRequest{
        bucket = bucketName
        key = "sample.csv"
    }) {
        it.body?.toInputStream()?.let { DataFrame.readCSV(it) }
    }
}

id,userid,username
1,1,User123
2,2,User456
3,3,User789
4,4,User200
5,5,User987


## Delete a bucket
Before deleting a bucket, you need to [empty the buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/delete-bucket.html).

The follow implementation is to list the bucket and deletes all items until the bucket is empty. Then delete the bucket.

In [47]:
import aws.sdk.kotlin.services.s3.deleteBucket
import aws.sdk.kotlin.services.s3.deleteObjects
import aws.sdk.kotlin.services.s3.listObjectsV2
import aws.sdk.kotlin.services.s3.model.Delete
import aws.sdk.kotlin.services.s3.model.ObjectIdentifier

val response = runBlocking {

    // empty bucket:
    var nextContinuationToken: String? = null
    do {
        val resp = s3Client.listObjectsV2 {
            bucket = bucketName
            continuationToken = nextContinuationToken
        }
        resp.contents?.let { contents ->
            s3Client.deleteObjects {
                bucket = bucketName
                delete = Delete {
                    objects = contents.map { ObjectIdentifier { key = it.key } }
                        .also { println("deleting $it") }
                }
            }
        }
        nextContinuationToken = resp.nextContinuationToken
    } while(nextContinuationToken != null)

    // delete bucket
    s3Client.deleteBucket {
        bucket = bucketName
    }
}

print(response)

deleting [ObjectIdentifier(eTag=null,key=sample.csv,lastModifiedTime=null,size=null,versionId=null)]
DeleteBucketResponse()